# MODFLOW API Paper

## Coupling of MODFLOW to PRMS

This notebook can be used to reproduce published results for the "Coupling of MODFLOW to PRMS" example, as reported in the MODFLOW 6 API paper (in progress).

## Supported operating systems
This example can be run on the following operating systems:

* linux
* macOS

## Prerequisites
To run the simulation, the following publicly available software and data are required:

* __libmf6.so__ (linux or MacOS) pre-compiled shared object (so) and available from https://github.com/MODFLOW-USGS/modflow6-nightly-build. The operating specific pre-compiled dll/so should be installed in the `bin` subdirectory at the same level as the subdirectory containing this Jupyter Notebook (`../bin/`). 
* __modflowapi__ is an extension to xmipy. __modflowapi__ including an implementation of its abstract methods. The source is available at https://github.com/MODFLOW-USGS/modflowapi and the module can be installed from PyPI using `pip install modflowapi`. __xmipy__ is an extension to bmipy including an implementation of its abstract methods and the source is available at https://github.com/Deltares/xmipy.
* __prms_surface__ and __prms_soil__ should be installed from the `csdms-stack` conda channel using `conda install -c csdms-stack prms_surface prms_soil`.
* __pymt__, __pymt_prms_surface__, and __pymt_prms_soil__ should be installed from the `csdms-stack` conda channel using `conda install -c csdms-stack pymt pymt_prms_surface pymt_prms_soil`.
* __flopy__ is a python package that can be used to build, run, and post-process MODFLOW 6 models. The source is available at https://github.com/modflowpy/flopy and the package can be installed from PyPI using `pip install flopy` or conda using `conda install flopy`.
* __netCDF4__ which can be installed using PyPI (`pip install netCDF4`) or conda (`conda install netCDF4`).

## Run the steady-state Sagehen MODFLOW 6 model

* Run the `sagehen-mf6-steady-state.ipynb` notebook prior to running this notebook to build the run the steady-state Sagehen model and create the initial head dataset needed by this notebook.

## Post-processing the results
After the full simulation is completed, model results can be post-processed using the `sagehen-postprocess-graphs.ipynb` and `sagehen-postprocess-maps.ipynb` Jupyter notebooks. The full simulation takes 8-10 hours to complete.

## Running the simulation

We start by importing the necessary packages:

In [ ]:
import os
import sys
import numpy as np
import netCDF4 as nc
from modflowapi import ModflowApi
from pymt.models import PRMSSurface, PRMSSoil
import prms_helpers # helper python script in the current directory. 
from pathlib import Path

### Model name and workspace

In [ ]:
name = "sagehenmodel"
ws = os.path.join(".", name)
print(Path(ws).exists(), os.getcwd())

In [ ]:
if sys.platform == "win32":
    mf6_dll = os.path.abspath('../bin/libmf6.dll')    
else:
    mf6_dll = os.path.abspath('../bin/libmf6.so')    

In [ ]:
init_ws = os.path.abspath(os.getcwd())
print(mf6_dll, os.path.isfile(mf6_dll), init_ws)

### Read weights

The weight matrix should have columns equal to the number of HRUs and rows equal to the number of UZF cells or number of SFR reaches.

_UZF weights_

In [ ]:
uz2 = np.load('weights.npz')
print(uz2['uzfw'].shape, uz2['sfrw'].shape)
uzfw = uz2['uzfw']

_SFR weights_

In [ ]:
sfrw = uz2['sfrw']

_Number of UZF cells at the top of the model_

In [ ]:
nuzf_infilt = uzfw.shape[0]
print("number of UZF cells at the top of the model {}".format(nuzf_infilt))

### Function to map HRU values to MODFLOW 6 values

In [ ]:
def hru2mf6(weights, values):
    return weights.dot(values)

### Run loosely coupled PRMS and MODFLOW 6 models

#### Initialize PRMS components

In [ ]:
# change to working directory
os.chdir(ws)

# create PRMS6 instance
config_surf = 'prms6_surf.control'
config_soil = 'prms6_soil.control'
# run_dir is current working directory here
run_dir = '.'
msurf = PRMSSurface()
msoil = PRMSSoil()
msurf.initialize(config_surf, run_dir)
msoil.initialize(config_soil, run_dir)

#### Calculate multipliers for PRMS internal variables

In [ ]:
m2ft = 3.28081
in2m = 1. / (12. * m2ft)

In [ ]:
fpth = os.path.join("prms_grid_v3-Copy1.nc")
ds = nc.Dataset(fpth)
hru_area = ds["hru_area"][:] # acres to m2
acre2m2 = 43560. / (m2ft * m2ft)

#### Create arrays to save results

In [ ]:
ntimes = int(msoil.end_time)
print("Number of days to simulate {}".format(ntimes))
time_out = np.zeros(
    ntimes,
    dtype=float,
)
ppt_out = np.zeros(
    (ntimes, hru_area.shape[0]),
    dtype=float,
)
actet_out = np.zeros(
    (ntimes, hru_area.shape[0]),
    dtype=float,
) 
potet_out = np.zeros(
    (ntimes, hru_area.shape[0]),
    dtype=float,
) 
soilinfil_out = np.zeros(
    (ntimes, hru_area.shape[0]),
    dtype=float,
) 
runoff_out = np.zeros(
    (ntimes, hru_area.shape[0]),
    dtype=float,
)        
interflow_out = np.zeros(
    (ntimes, hru_area.shape[0]),
    dtype=float,
) 

#### Initialize MODFLOW 6

In [ ]:
# create MODFLOW 6 model instance
mf6_config_file = os.path.join(ws, 'mfsim.nam')
mf6 = ModflowApi(mf6_dll)

# initialize the MODFLOW 6 model
mf6.initialize(mf6_config_file)

# MODFLOW 6 time loop
current_time = mf6.get_current_time()
end_time = mf6.get_end_time()
print(f'MF current_time: {current_time}, PRMS current_time: {msoil.time}')
print(f'MF end_time: {end_time}, PRMS end_time: {msoil.end_time}')

#### Get pointers to MODFLOW 6 variables

In [ ]:
# get pointer to UZF variables
infilt = mf6.get_value_ptr(
    mf6.get_var_address("SINF", name.upper(), "UZF-1")
)
pet = mf6.get_value_ptr(
    mf6.get_var_address("PET", name.upper(), "UZF-1")
)

# get pointer to SFR variables
runoff = mf6.get_value_ptr(
    mf6.get_var_address("RUNOFF", name.upper(), "SFR-1")
)

# print array sizes
print("data shapes\n\tINFILT {}\n\tPET {}\n\tRUNOFF {}".format(infilt.shape, pet.shape, runoff.shape))

#### Run the models

In [ ]:
# model time loop
idx = 0
tmax = 5844  # full simulation period = 5844
while current_time < end_time:
    if current_time > tmax:
        break
        
    # write info to the screen
    if current_time % 365. == 0.:
        stdout_str = "{:d}\n".format(1980 + int(current_time/365))
        if idx > 0:
            stdout_str = "\n" + stdout_str
    else:
        stdout_str = "."
    sys.stdout.write(stdout_str)

    # run PRMS
    prms_helpers.update_coupled(msurf, msoil)

    # get PRMS ppt for output
    hru_ppt = msurf.get_value('hru_ppt')
    
    # get PRMS soil recharge rate
    soil_infil = msoil.get_value('ssres_in')
    soil_infil += msoil.get_value('pref_flow_infil')

    # get prms groundwater recharge
    ssr_to_gw = msoil.get_value("ssr_to_gw")
    soil_to_gw = msoil.get_value("soil_to_gw")
    recharge = (ssr_to_gw + soil_to_gw)

    # get prms unsatisfied potential et for UZF and groundwater
    potet = msurf.get_value("potet")
    actet = msoil.get_value("hru_actet")
    unused_pet = (potet - actet)

    # get prms runoff and interflow for output
    sroff = msoil.get_value('sroff') * in2m * hru_area * acre2m2    
    interflow = msoil.get_value("ssres_flow") * in2m * hru_area * acre2m2

    # get prms runoff 
    prms_ro = msoil.get_value("sroff") 
    prms_ro += msoil.get_value("ssres_flow")
    prms_ro *= in2m * hru_area * acre2m2

    # save PRMS results (converted to m3/d) 
    time_out[idx] = msoil.time
    ppt_out[idx, :] = hru_ppt * in2m * hru_area * acre2m2
    potet_out[idx, :] = potet * in2m * hru_area * acre2m2
    actet_out[idx, :] = actet * in2m * hru_area * acre2m2
    soilinfil_out[idx, :] = soil_infil * in2m * hru_area * acre2m2
    runoff_out[idx,:] = sroff
    interflow_out[idx,:] = interflow    
    
    # map runoff to SFR
    v = hru2mf6(sfrw, prms_ro)
    runoff[:] = v

    # map groundwater recharge to MODFLOW
    v = hru2mf6(uzfw, recharge) * in2m
    infilt[:nuzf_infilt] = v

    # map unused pet to MODFLOW
    v = hru2mf6(uzfw, unused_pet) * in2m
    pet[:nuzf_infilt] = v
    
    # run MODFLOW 6
    mf6.update()

    # update time
    current_time = mf6.get_current_time()

    # increment time step counter
    idx += 1

### Finalize models

In [ ]:
# cleanup
try:
    mf6.finalize()
    msurf.finalize()
    msoil.finalize()
    success = True
except:
    raise RuntimeError

# change back to the starting directory
os.chdir(init_ws)

#### Save PRMS output

In [ ]:
fpth = "sagehenmodel/output/prms_output.npz"
np.savez_compressed(
    fpth,
    time=time_out, 
    ppt=ppt_out, 
    potet=potet_out, 
    actet=actet_out, 
    infil=soilinfil_out,
    runoff=runoff_out,
    interflow=interflow_out,
)